In [1]:
%cd /content/drive/MyDrive/Colab/instate_v2/

/content/drive/MyDrive/Colab/instate_v2


# Extract last names
In the current given dataset, we have states that have names with english and states with non-english.

In [9]:
import pandas as pd

## Processing states has names in english

In [ ]:
# total states
!ls -ltr data/clean | grep -v "_ln.csv" | grep "csv" | wc -l

15


In [ ]:
# below are the states that have names in english
!ls -ltr data/clean | grep -v "_ln.csv"

total 18762410
-rw------- 1 root root 7063645140 Apr 25 23:26 andhra.csv
-rw------- 1 root root  220875368 Apr 25 23:58 arunachal.csv
-rw------- 1 root root   30437784 Apr 28 07:59 dadra.csv
-rw------- 1 root root   29861386 Apr 28 08:04 daman.csv
-rw------- 1 root root 4655371427 May 24 22:40 delhi.csv
-rw------- 1 root root  304924618 May 24 22:47 goa.csv
-rw------- 1 root root   45260206 May 24 22:50 jk.csv
-rw------- 1 root root  458125766 May 24 22:51 manipur.csv
-rw------- 1 root root  438116162 May 24 22:53 meghalaya.csv
-rw------- 1 root root  191618060 May 24 22:54 mizoram.csv
-rw------- 1 root root  262045318 May 24 22:55 nagaland.csv
-rw------- 1 root root  316673687 May 24 22:56 puducherry.csv
-rw------- 1 root root   89363047 May 24 23:19 sikkim.csv
-rw------- 1 root root 5046609351 May 24 23:43 kerala.csv
-rw------- 1 root root   58581352 Jun 18 18:58 andaman.csv


In [ ]:
# get size of above files
!ls -ltr data/clean | grep -v "_ln.csv" | awk '{print $9}' | xargs -I % du -sh data/clean/%

6.6G	data/clean/andhra.csv
211M	data/clean/arunachal.csv
30M	data/clean/dadra.csv
29M	data/clean/daman.csv
4.4G	data/clean/delhi.csv
291M	data/clean/goa.csv
44M	data/clean/jk.csv
437M	data/clean/manipur.csv
418M	data/clean/meghalaya.csv
183M	data/clean/mizoram.csv
250M	data/clean/nagaland.csv
303M	data/clean/puducherry.csv
86M	data/clean/sikkim.csv
4.8G	data/clean/kerala.csv
56M	data/clean/andaman.csv


In [ ]:
# Function to normalize names and return a set of possible last names
def get_possible_last_names(name):
    parts = name.split(' ')
    return set(parts)


# Helper function to extract last names and filter out invalid ones
def extract_valid_last_name(name1, name2):
    # Split the names by space
    parts1 = name1.split()
    parts2 = name2.split()

    # Extract the last names
    last_name1 = parts1[-1] if len(parts1) > 1 else ''
    last_name2 = parts2[-1] if len(parts2) > 1 else ''

    # Check if the last names are valid (not a single or double letter) and match
    if last_name1 == last_name2 and len(last_name1) > 2:
        return last_name1
    else:
        return None


def process_lastnames(file, state, path="data/final"):
    df = pd.read_csv(file, encoding='utf-8', on_bad_lines='skip')

    # drop na values in columns elector_name and father_or_husband_name
    df = df.dropna(subset=['elector_name', 'father_or_husband_name'])

    # Normalize the names
    df['last_names1'] = df['elector_name'].apply(get_possible_last_names)
    df['last_names2'] = df['father_or_husband_name'].apply(get_possible_last_names)

    # Find matching rows by checking for intersections in the sets of last names
    df = df[df.apply(lambda row: len(row['last_names1'].intersection(row['last_names2'])) > 0, axis=1)]
    # extract last name
    df['last_name'] = df.apply(lambda row: next(iter(row['last_names1'].intersection(row['last_names2'])),''), axis=1)
    # Filter out the rows where last_name is None
    df = df.dropna(subset=['last_name'])
    # Convert the last_name column to lowercase
    df['last_name'] = df['last_name'].str.lower()
    # Remove any last names less than 3
    df = df[df['last_name'].str.len() >= 3]
    # Get the value counts of the last names
    last_name_counts = df['last_name'].value_counts()
    # Convert the Series to a DataFrame
    last_name_counts_df = last_name_counts.reset_index()
    last_name_counts_df.columns = ['last_name', 'count']
    # Add the state column
    last_name_counts_df['state'] = state
    # Save the DataFrame to a CSV file
    last_name_counts_df.to_csv(f'{path}/{state}_ln.csv', index=False)

In [ ]:
states_w_eng_names = !ls -ltr data/clean | grep -v "_ln.csv" | grep "csv" | awk '{print $9}'

In [ ]:
states_w_eng_names

['andhra.csv',
 'arunachal.csv',
 'dadra.csv',
 'daman.csv',
 'delhi.csv',
 'goa.csv',
 'jk.csv',
 'manipur.csv',
 'meghalaya.csv',
 'mizoram.csv',
 'nagaland.csv',
 'puducherry.csv',
 'sikkim.csv',
 'kerala.csv',
 'andaman.csv']

In [ ]:
for state in states_w_eng_names:
  state = state.split('.')[0]
  print(f"starting processing for {state}")
  process_lastnames(f'data/clean/{state}.csv', state)
  print(f"finished processing for {state}")

starting processing for andhra
finished processing for andhra
starting processing for arunachal
finished processing for arunachal
starting processing for dadra
finished processing for dadra
starting processing for daman
finished processing for daman
starting processing for delhi


<ipython-input-45-67fe629f8da6>:25: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding='utf-8', on_bad_lines='skip')


finished processing for delhi
starting processing for goa
finished processing for goa
starting processing for jk
finished processing for jk
starting processing for manipur


<ipython-input-45-67fe629f8da6>:25: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding='utf-8', on_bad_lines='skip')


finished processing for manipur
starting processing for meghalaya
finished processing for meghalaya
starting processing for mizoram


...

finished processing for mizoram
starting processing for nagaland


<ipython-input-45-67fe629f8da6>:25: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding='utf-8', on_bad_lines='skip')


finished processing for nagaland
starting processing for puducherry
finished processing for puducherry
starting processing for sikkim
finished processing for sikkim
starting processing for kerala


<ipython-input-45-67fe629f8da6>:25: DtypeWarning: Columns (14,16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding='utf-8', on_bad_lines='skip')


finished processing for kerala
starting processing for andaman
finished processing for andaman


## Processing names that are not in english

In [ ]:
# below are the states that have names in english
!ls -ltr data/non_english/ | grep -v "_ln"

total 382024519
-rw------- 1 root root 30371244722 May 24 23:04 west_bengal.csv
-rw------- 1 root root 94178659023 May 27 22:45 uttar_pradesh.csv
-rw------- 1 root root 30799628245 May 27 22:45 rajasthan.csv
-rw------- 1 root root 13019509069 May 27 22:45 punjab.csv
-rw------- 1 root root 24524158426 May 27 22:45 odisha.csv
-rw------- 1 root root 30171537617 May 27 22:45 madhya_pradesh.csv
-rw------- 1 root root  5786439167 May 27 22:45 karnataka.csv
-rw------- 1 root root 10790608302 May 27 22:45 haryana.csv
-rw------- 1 root root 16324706456 May 27 22:45 gujarat.csv
-rw------- 1 root root 64123876606 May 27 22:45 bihar.csv
-rw------- 1 root root  6385059843 May 27 22:45 assam.csv
-rw------- 1 root root  4881172872 May 27 22:45 uttarakhand.csv
-rw------- 1 root root  2263448186 May 27 22:45 tripura.csv
-rw------- 1 root root  8522263084 May 27 22:45 telangana.csv
-rw------- 1 root root 32063522554 May 27 22:45 tamil_nadu.csv
-rw------- 1 root root 13816008964 May 27 22:45 jharkhand.cs

In [ ]:
# total states
!ls -ltr data/non_english/ | grep -v "_ln" | grep "csv" | wc -l

18


In [ ]:
# get size of the files
!ls -ltr data/non_english/ | grep -v "_ln" | awk '{print $9}' | xargs -I % du -sh data/non_english/%

29G	data/non_english/west_bengal.csv
88G	data/non_english/uttar_pradesh.csv
29G	data/non_english/rajasthan.csv
13G	data/non_english/punjab.csv
23G	data/non_english/odisha.csv
29G	data/non_english/madhya_pradesh.csv
5.4G	data/non_english/karnataka.csv
11G	data/non_english/haryana.csv
16G	data/non_english/gujarat.csv
60G	data/non_english/bihar.csv
6.0G	data/non_english/assam.csv
4.6G	data/non_english/uttarakhand.csv
2.2G	data/non_english/tripura.csv
8.0G	data/non_english/telangana.csv
30G	data/non_english/tamil_nadu.csv
13G	data/non_english/jharkhand.csv
2.6G	data/non_english/himachal_pradesh.csv
362M	data/non_english/chandigarh.csv


In [ ]:
# sort based on size
!ls -ltr data/non_english/ | grep -v "_ln" | awk '{print $9}' |  xargs -I % du -sh --block-size=G data/non_english/% | sort -k1,1 -h

1G	data/non_english/chandigarh.csv
3G	data/non_english/himachal_pradesh.csv
3G	data/non_english/tripura.csv
5G	data/non_english/uttarakhand.csv
6G	data/non_english/assam.csv
6G	data/non_english/karnataka.csv
8G	data/non_english/telangana.csv
11G	data/non_english/haryana.csv
13G	data/non_english/jharkhand.csv
13G	data/non_english/punjab.csv
16G	data/non_english/gujarat.csv
23G	data/non_english/odisha.csv
29G	data/non_english/madhya_pradesh.csv
29G	data/non_english/rajasthan.csv
29G	data/non_english/west_bengal.csv
30G	data/non_english/tamil_nadu.csv
60G	data/non_english/bihar.csv
88G	data/non_english/uttar_pradesh.csv


In [ ]:
# with 84 GB RAM we can fit till 16G
# so get files that are less than 20G
!ls -ltr data/non_english/ | grep -v "_ln" | awk '{print $9}' |  xargs -I % du -sh --block-size=G data/non_english/% \
  | sort -k1,1 -h | awk '$1 ~ /G$/ && substr($1, 1, length($1)-1) + 0 < 20'

1G	data/non_english/chandigarh.csv
3G	data/non_english/himachal_pradesh.csv
3G	data/non_english/tripura.csv
5G	data/non_english/uttarakhand.csv
6G	data/non_english/assam.csv
6G	data/non_english/karnataka.csv
8G	data/non_english/telangana.csv
11G	data/non_english/haryana.csv
13G	data/non_english/jharkhand.csv
13G	data/non_english/punjab.csv
16G	data/non_english/gujarat.csv


In [ ]:
states_non_eng_lt_20g = !ls -ltr data/non_english/ | grep -v "_ln" | awk '{print $9}' |  xargs -I % du -sh --block-size=G data/non_english/% \
  | sort -k1,1 -h | awk '$1 ~ /G$/ && substr($1, 1, length($1)-1) + 0 < 20' | awk '{print $2}'

In [ ]:
states_non_eng_lt_20g

['data/non_english/chandigarh.csv',
 'data/non_english/himachal_pradesh.csv',
 'data/non_english/tripura.csv',
 'data/non_english/uttarakhand.csv',
 'data/non_english/assam.csv',
 'data/non_english/karnataka.csv',
 'data/non_english/telangana.csv',
 'data/non_english/haryana.csv',
 'data/non_english/jharkhand.csv',
 'data/non_english/punjab.csv',
 'data/non_english/gujarat.csv']

In [ ]:
# trim data/non_english
for state in states_non_eng_lt_20g:
  state = state.split('.')[0].split('/')[-1]
  print(f"starting processing for {state}")
  process_lastnames(f'data/non_english/{state}.csv', state, path="data/final/non_eng")
  print(f"finished processing for {state}")

starting processing for chandigarh
finished processing for chandigarh
starting processing for himachal_pradesh


<ipython-input-70-71479c40cc55>:25: DtypeWarning: Columns (8,9,10,16,17,18,29,30,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding='utf-8', on_bad_lines='skip')


finished processing for himachal_pradesh
starting processing for tripura


<ipython-input-70-71479c40cc55>:25: DtypeWarning: Columns (0,10,18,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding='utf-8', on_bad_lines='skip')


finished processing for tripura
starting processing for uttarakhand


...

finished processing for jharkhand
starting processing for punjab


<ipython-input-70-71479c40cc55>:25: DtypeWarning: Columns (8,10,14,17,20,28,30,33,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding='utf-8', on_bad_lines='skip')


finished processing for punjab
starting processing for gujarat


<ipython-input-70-71479c40cc55>:25: DtypeWarning: Columns (0,15,17,31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding='utf-8', on_bad_lines='skip')


finished processing for gujarat


In [ ]:
states_non_eng_gt_20g = !ls -ltr data/non_english/ | grep -v "_ln" | awk '{print $9}' |  xargs -I % du -sh --block-size=G data/non_english/% \
  | sort -k1,1 -h | awk '$1 ~ /G$/ && substr($1, 1, length($1)-1) + 0 > 20'

In [ ]:
states_non_eng_gt_20g

['23G\tdata/non_english/odisha.csv',
 '29G\tdata/non_english/madhya_pradesh.csv',
 '29G\tdata/non_english/rajasthan.csv',
 '29G\tdata/non_english/west_bengal.csv',
 '30G\tdata/non_english/tamil_nadu.csv',
 '60G\tdata/non_english/bihar.csv',
 '88G\tdata/non_english/uttar_pradesh.csv']

In [ ]:
import math
math.ceil(int("23G\tdata/non_english/odisha.csv".split('\t')[0].split('G')[0])/15)

2

In [ ]:
def merge_agg(files, state):
    dataframes = [pd.read_csv(file) for file in files]
    combined_df = pd.concat(dataframes, ignore_index=True)

    aggregated_data = combined_df.groupby('last_name')['count'].sum().reset_index()
    aggregated_data.columns = ['last_name', 'count']
    aggregated_data['state'] = state

    aggregated_data.to_csv(f'data/final/non_eng/{state}_ln.csv', index=False)

In [ ]:
for state in states_non_eng_gt_20g:
  size = state.split('\t')[0]
  split_size = math.ceil(int(size.split('G')[0])/15)
  file_path = state.split('\t')[1]
  state = file_path.split('.')[0].split('/')[-1]
  print(f"starting processing for {state} with size {size}, split size {split_size}")
  # split file based on split_size
  !split -n $split_size $file_path data/non_eng_split/
  # get split files
  split_files = !find data/non_eng_split/ -type f
  # process each split file
  file_no = 1
  final_files = []
  for split_file in split_files:
    if 'headers' in split_file:
      continue
    print(split_file)
    print(state)
    # get headers
    if file_no == 1:
      !head -1 $split_file > ./data/non_eng_split/headers.csv
    else:
      # remove first line
      !sed -i '1d' $split_file
      # add headers to first line
      !cat ./data/non_eng_split/headers.csv $split_file > ./data/non_eng_split/tmp.csv
      !mv ./data/non_eng_split/tmp.csv $split_file
    # remove last line
    !sed -i '$d' $split_file
    file_name = split_file.split('/')[-1]
    process_lastnames(split_file, file_name, path="data/non_eng_split")
    file_no = file_no + 1
    final_files.append(f"{split_file}_ln.csv")
    print()
  merge_agg(final_files, state)
  # remove all files in non_eng_split
  !rm -rf data/non_eng_split/*
  print(f"finished processing for {state}")

starting processing for odisha with size 23G, split size 2
data/non_eng_split/aa
odisha

data/non_eng_split/ab
odisha


<ipython-input-31-71479c40cc55>:25: DtypeWarning: Columns (8,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding='utf-8', on_bad_lines='skip')



finished processing for odisha
starting processing for madhya_pradesh with size 29G, split size 2
data/non_eng_split/aa
madhya_pradesh


<ipython-input-31-71479c40cc55>:25: DtypeWarning: Columns (0,14,17,31,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding='utf-8', on_bad_lines='skip')



data/non_eng_split/ab
madhya_pradesh


...

<ipython-input-31-71479c40cc55>:25: DtypeWarning: Columns (0,5,10,14,15,16,17,18,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding='utf-8', on_bad_lines='skip')



data/non_eng_split/ae
uttar_pradesh


<ipython-input-31-71479c40cc55>:25: DtypeWarning: Columns (0,5,10,14,15,16,17,18,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding='utf-8', on_bad_lines='skip')



data/non_eng_split/af
uttar_pradesh


<ipython-input-31-71479c40cc55>:25: DtypeWarning: Columns (0,5,10,14,16,17,31,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding='utf-8', on_bad_lines='skip')


## Now convert all non-english languages to english

In [3]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00


In [4]:
import getpass
from openai import OpenAI

OPENAI_API_KEY = getpass.getpass("OpenAI API key: ")
client = OpenAI(api_key=OPENAI_API_KEY)

OpenAI API key: ··········


In [5]:
import os
import pandas as pd
import csv
import time
import requests
import json

def generate(prompt):
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "user", "content": prompt}
    ]
  )
  return response.choices[0].message.content

# Function to divide names into chunks
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


# Function to get already processed names
def get_processed_names(file):
    if os.path.exists(file):
        with open(file, 'r') as f:
            reader = csv.DictReader(f)
            return {row['Name'] for row in reader}
    else:
        # create file
        with open(file, 'w') as f:
            writer = csv.DictWriter(f, fieldnames=['Name', 'Translated'])
            writer.writeheader()
        return set()


def transliterate(file):
  df = pd.read_csv(file)
  unique_names = df['last_name'].unique()
  print(f"total names: {len(unique_names)}")
  state = file.split('.')[0]
  translated_file = f"{state}_trans.csv"

  # Get already processed names
  processed_names = get_processed_names(translated_file)
  print(f"till now processed: {len(processed_names)}")

  prompt = """
  Transliterate the given text to English. Do not translate.
  Respond in json format - {"key": "value"} where value is the transliterated text of key.
  Below is the example for language hindi.

  input:  'चंचला', 'रानी', 'दास', 'विकास'
  output:  {'चंचला': 'Chanchala', 'रानी': 'Rani', 'दास': 'Das','विकास': 'Vikas'}

  """

  with open(translated_file, 'a', newline='') as csvfile:
      fieldnames = ['Name', 'Translated']
      writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

      # If file is empty write the header
      if csvfile.tell() == 0:
          writer.writeheader()

      c = 0
      # Iterate over names in chunks
      main_chunk = []
      for chunk in chunks(list(unique_names), 50):
          if c!=0 and c%50 == 0:
              print(f"done processing {c * 50}")
          c = c + 1

          # Skip names that have been processed
          chunk = [name for name in chunk if name not in processed_names]
          if not chunk:
              continue

          # Append the chunk to the main chunk
          main_chunk.extend(chunk)
          # if length of main chunk is less 20, add next chunk
          if len(main_chunk) < 20:
              continue

          fail = False
          while True:
              try:
                  # Make the API call
                  # print(prompt + ', '.join(chunk))
                  chat_response = generate(prompt + ', '.join(main_chunk))
                  #print(chat_response)
              except Exception as e:
                  print(e)
                  time.sleep(2)  # Sleep for 2 seconds and then retry
                  fail = True
                  #continue

              break  # If no errors, break the loop

          if fail:
            continue
          try:
            if chat_response:
              results = json.loads(chat_response)
              for name in results:
                  writer.writerow({'Name': name, 'Translated': results[name]})
              csvfile.flush()  # Ensure data is written after each batch
          except Exception as e:
            print(e)
          time.sleep(2)  # Sleep for second
          main_chunk = []

def merge_df(file1, file2):
  df1 = pd.read_csv(file1)
  df2 = pd.read_csv(file2)
  df2 = df2.rename(columns={'Name': 'last_name'})
  df3 = pd.merge(df1, df2, on='last_name', how='outer')
  df3.to_csv(f"{file1.split('.')[0]}_processed.csv", index=False)

In [6]:
%cd data/final/non_eng

/content/drive/MyDrive/Colab/instate_v2/data/final/non_eng


In [7]:
!ls -ltr *_ln.csv

-rw------- 1 root root  198754 Jun 18 21:49 himachal_pradesh_ln.csv
-rw------- 1 root root  141842 Jun 18 21:50 tripura_ln.csv
-rw------- 1 root root  415775 Jun 18 21:53 uttarakhand_ln.csv
-rw------- 1 root root  443221 Jun 18 21:57 assam_ln.csv
-rw------- 1 root root 2307232 Jun 18 22:00 karnataka_ln.csv
-rw------- 1 root root 5011806 Jun 18 22:06 telangana_ln.csv
-rw------- 1 root root  537318 Jun 18 22:13 haryana_ln.csv
-rw------- 1 root root 1308216 Jun 18 22:23 jharkhand_ln.csv
-rw------- 1 root root  396258 Jun 18 22:32 punjab_ln.csv
-rw------- 1 root root   39525 Jun 18 22:41 gujarat_ln.csv
-rw------- 1 root root  192120 Jun 22 00:01 chandigarh_ln.csv
-rw------- 1 root root 2003685 Jun 22 01:00 odisha_ln.csv
-rw------- 1 root root 2783105 Jun 22 01:36 madhya_pradesh_ln.csv
-rw------- 1 root root  705463 Jun 22 02:09 rajasthan_ln.csv
-rw------- 1 root root 2586427 Jun 22 02:55 west_bengal_ln.csv
-rw------- 1 root root 1064265 Jun 22 03:31 tamil_nadu_ln.csv
-rw------- 1 root root

In [8]:
non_eng_files = !ls -ltr *_ln.csv | awk '{print $9}'

In [9]:
non_eng_files

['himachal_pradesh_ln.csv',
 'tripura_ln.csv',
 'uttarakhand_ln.csv',
 'assam_ln.csv',
 'karnataka_ln.csv',
 'telangana_ln.csv',
 'haryana_ln.csv',
 'jharkhand_ln.csv',
 'punjab_ln.csv',
 'gujarat_ln.csv',
 'chandigarh_ln.csv',
 'odisha_ln.csv',
 'madhya_pradesh_ln.csv',
 'rajasthan_ln.csv',
 'west_bengal_ln.csv',
 'tamil_nadu_ln.csv',
 'bihar_ln.csv',
 'uttar_pradesh_ln.csv']

In [10]:
for file in non_eng_files:
  print(f"processing {file}")
  transliterate(file)
  merge_df(file, f"{file.split('.')[0]}_trans.csv")
  print(f"finished processing {file}")

processing himachal_pradesh_ln.csv
total names: 5488
till now processed: 5516
done processing 2500
done processing 5000
finished processing himachal_pradesh_ln.csv
processing tripura_ln.csv
total names: 4687
till now processed: 4701
done processing 2500
finished processing tripura_ln.csv
processing uttarakhand_ln.csv
total names: 12786
till now processed: 12832
done processing 2500
done processing 5000
done processing 7500
done processing 10000
done processing 12500
finished processing uttarakhand_ln.csv
processing assam_ln.csv
total names: 16367
till now processed: 16480
done processing 2500
done processing 5000
done processing 7500
done processing 10000
done processing 12500
done processing 15000
finished processing assam_ln.csv
processing karnataka_ln.csv
total names: 67679
till now processed: 68171
done processing 2500
done processing 5000
done processing 7500
done processing 10000
done processing 12500
done processing 15000
done processing 17500
done processing 20000
done processi

In [2]:
!pwd

/content/drive/MyDrive/Colab/instate_v2


In [3]:
%cd data/final

/content/drive/MyDrive/Colab/instate_v2/data/final


In [18]:
!pwd

/content/drive/MyDrive/Colab/instate_v2/data/final


In [5]:
!ls -ltr non_eng/*processed.csv

-rw------- 1 root root  251393 Jul  3 17:23 non_eng/himachal_pradesh_ln_processed.csv
-rw------- 1 root root  190130 Jul  3 17:23 non_eng/tripura_ln_processed.csv
-rw------- 1 root root  540676 Jul  3 17:23 non_eng/uttarakhand_ln_processed.csv
-rw------- 1 root root  613714 Jul  3 17:23 non_eng/assam_ln_processed.csv
-rw------- 1 root root 3131897 Jul  3 17:23 non_eng/karnataka_ln_processed.csv
-rw------- 1 root root 6739893 Jul  3 17:24 non_eng/telangana_ln_processed.csv
-rw------- 1 root root  723629 Jul  3 17:24 non_eng/haryana_ln_processed.csv
-rw------- 1 root root 1739123 Jul  3 17:25 non_eng/jharkhand_ln_processed.csv
-rw------- 1 root root  541623 Jul  3 17:25 non_eng/punjab_ln_processed.csv
-rw------- 1 root root   53748 Jul  3 17:25 non_eng/gujarat_ln_processed.csv
-rw------- 1 root root  253527 Jul  3 17:25 non_eng/chandigarh_ln_processed.csv
-rw------- 1 root root 2807809 Jul  3 17:31 non_eng/odisha_ln_processed.csv
-rw------- 1 root root 3563050 Jul  3 17:34 non_eng/madhya

In [6]:
!head -5 non_eng/himachal_pradesh_ln_processed.csv

last_name,count,state,Translated
सिंह,160827.0,himachal_pradesh,Singh
सिंह,160827.0,himachal_pradesh,Singh
सिंह,160827.0,himachal_pradesh,Singh
सिंह,160827.0,himachal_pradesh,Singh


In [8]:
!head -5 andhra_ln.csv

last_name,count,state
shaik,423139,andhra
reddy,165388,andhra
rao,102433,andhra
areddy,49538,andhra


## Move proceesed files from non_eng to current folder

In [20]:
processed = !ls -ltr non_eng/*processed.csv | awk '{print $9}'

In [21]:
processed

['non_eng/himachal_pradesh_ln_processed.csv',
 'non_eng/tripura_ln_processed.csv',
 'non_eng/uttarakhand_ln_processed.csv',
 'non_eng/assam_ln_processed.csv',
 'non_eng/karnataka_ln_processed.csv',
 'non_eng/telangana_ln_processed.csv',
 'non_eng/haryana_ln_processed.csv',
 'non_eng/jharkhand_ln_processed.csv',
 'non_eng/punjab_ln_processed.csv',
 'non_eng/gujarat_ln_processed.csv',
 'non_eng/chandigarh_ln_processed.csv',
 'non_eng/odisha_ln_processed.csv',
 'non_eng/madhya_pradesh_ln_processed.csv',
 'non_eng/rajasthan_ln_processed.csv',
 'non_eng/west_bengal_ln_processed.csv',
 'non_eng/tamil_nadu_ln_processed.csv',
 'non_eng/bihar_ln_processed.csv',
 'non_eng/uttar_pradesh_ln_processed.csv']

In [22]:
def move_processed(csv_file):
  print(f"moving {csv_file}")
  to_file = csv_file.replace('non_eng/', '')
  to_file = to_file.replace('_processed.csv', '.csv')
  df = pd.read_csv(csv_file)
  # drop last_name column
  df = df.drop('last_name', axis=1)
  # rename Translated to last_name
  df = df.rename(columns={'Translated': 'last_name'})
  # write it to csv file
  df.to_csv(to_file, index=False)

In [23]:
for csv_file in processed:
  move_processed(csv_file)

moving non_eng/himachal_pradesh_ln_processed.csv
moving non_eng/tripura_ln_processed.csv
moving non_eng/uttarakhand_ln_processed.csv
moving non_eng/assam_ln_processed.csv
moving non_eng/karnataka_ln_processed.csv
moving non_eng/telangana_ln_processed.csv
moving non_eng/haryana_ln_processed.csv
moving non_eng/jharkhand_ln_processed.csv
moving non_eng/punjab_ln_processed.csv
moving non_eng/gujarat_ln_processed.csv
moving non_eng/chandigarh_ln_processed.csv
moving non_eng/odisha_ln_processed.csv
moving non_eng/madhya_pradesh_ln_processed.csv
moving non_eng/rajasthan_ln_processed.csv
moving non_eng/west_bengal_ln_processed.csv
moving non_eng/tamil_nadu_ln_processed.csv
moving non_eng/bihar_ln_processed.csv
moving non_eng/uttar_pradesh_ln_processed.csv


## Cleanup duplicates and remove null values

In [29]:
states = !ls -ltr *_ln.csv | awk '{print $9}'

In [33]:
states

['andhra_ln.csv',
 'arunachal_ln.csv',
 'dadra_ln.csv',
 'daman_ln.csv',
 'delhi_ln.csv',
 'goa_ln.csv',
 'jk_ln.csv',
 'manipur_ln.csv',
 'meghalaya_ln.csv',
 'mizoram_ln.csv',
 'nagaland_ln.csv',
 'puducherry_ln.csv',
 'sikkim_ln.csv',
 'kerala_ln.csv',
 'andaman_ln.csv',
 'tripura_ln.csv',
 'himachal_pradesh_ln.csv',
 'uttarakhand_ln.csv',
 'assam_ln.csv',
 'karnataka_ln.csv',
 'telangana_ln.csv',
 'jharkhand_ln.csv',
 'haryana_ln.csv',
 'punjab_ln.csv',
 'gujarat_ln.csv',
 'chandigarh_ln.csv',
 'odisha_ln.csv',
 'madhya_pradesh_ln.csv',
 'rajasthan_ln.csv',
 'west_bengal_ln.csv',
 'tamil_nadu_ln.csv',
 'bihar_ln.csv',
 'uttar_pradesh_ln.csv']

In [31]:
def cleanup(csv_file):
  df = pd.read_csv(csv_file)
  df = df.drop_duplicates(subset=['last_name','state','count'])
  df = df.dropna()
  # group by last_name and sum the counts
  df = df.groupby(['last_name','state'])['count'].sum().reset_index()
  df = df.sort_values(by=['count'], ascending=False)
  df.to_csv(csv_file, index=False)

In [34]:
for state in states:
  cleanup(state)

## Merge all states to one file

In [35]:
dfs = []
for state in states:
  df = pd.read_csv(state)
  dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)
merged_df.to_csv('all_states.csv', index=False)

## Map states to languages

In [36]:
!pwd

/content/drive/MyDrive/Colab/instate_v2/data/final


In [37]:
!wget https://github.com/appeler/instate/blob/main/instate/data/state_to_languages.csv?raw=true -O state_to_languages.csv

--2024-07-04 22:26:36--  https://github.com/appeler/instate/blob/main/instate/data/state_to_languages.csv?raw=true
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/appeler/instate/raw/main/instate/data/state_to_languages.csv [following]
--2024-07-04 22:26:36--  https://github.com/appeler/instate/raw/main/instate/data/state_to_languages.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/appeler/instate/main/instate/data/state_to_languages.csv [following]
--2024-07-04 22:26:36--  https://raw.githubusercontent.com/appeler/instate/main/instate/data/state_to_languages.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubuserconte

In [38]:
import pandas as pd
df = pd.read_csv('state_to_languages.csv')
df.head()

,state,official_languages,addl_official_languages,most_spoken_lang,second_most_spoken_lang,third_most_spoken_lang,fourth_most_spoken_lang,fifth_most_spoken_lang
0,Andaman and Nicobar Islands,"Hindi, English",NaN,Bengali,Hindi,Tamil,Telugu,Malayalam
1,Andhra Pradesh,Telugu,Urdu,Telugu,Urdu,Tamil,Chenchu,Kannada
2,Arunachal Pradesh,English,NaN,Nishi,Adi,Apatani,Tagin,Hindi
3,Assam,Assamese,"Bengali, Bodo",Assamese,Bengali,Bodo,Hindi,Nepali
4,Bihar,Hindi,Urdu,Hindi (including Bihari languages),Maithili,Urdu,Bengali,Santali


In [48]:
lang_states = df['state'].str.lower().to_list()

In [49]:
lang_states

['andaman and nicobar islands',
 'andhra pradesh',
 'arunachal pradesh',
 'assam',
 'bihar',
 'chandigarh',
 'chhattisgarh',
 'dadra and nagar haveli and daman and diu',
 'delhi',
 'goa',
 'gujarat',
 'haryana',
 'himachal pradesh',
 'jammu and kashmir and ladakh',
 'jharkhand',
 'karnataka',
 'kerala',
 'lakshadweep',
 'madhya pradesh',
 'maharashtra',
 'manipur',
 'meghalaya',
 'mizoram',
 'nagaland',
 'odisha',
 'puducherry',
 'punjab',
 'rajasthan',
 'sikkim',
 'tamil nadu',
 'telangana',
 'tripura',
 'uttar pradesh',
 'uttarakhand',
 'west bengal']

In [46]:
states

['andhra_ln.csv',
 'arunachal_ln.csv',
 'dadra_ln.csv',
 'daman_ln.csv',
 'delhi_ln.csv',
 'goa_ln.csv',
 'jk_ln.csv',
 'manipur_ln.csv',
 'meghalaya_ln.csv',
 'mizoram_ln.csv',
 'nagaland_ln.csv',
 'puducherry_ln.csv',
 'sikkim_ln.csv',
 'kerala_ln.csv',
 'andaman_ln.csv',
 'tripura_ln.csv',
 'himachal_pradesh_ln.csv',
 'uttarakhand_ln.csv',
 'assam_ln.csv',
 'karnataka_ln.csv',
 'telangana_ln.csv',
 'jharkhand_ln.csv',
 'haryana_ln.csv',
 'punjab_ln.csv',
 'gujarat_ln.csv',
 'chandigarh_ln.csv',
 'odisha_ln.csv',
 'madhya_pradesh_ln.csv',
 'rajasthan_ln.csv',
 'west_bengal_ln.csv',
 'tamil_nadu_ln.csv',
 'bihar_ln.csv',
 'uttar_pradesh_ln.csv']

In [47]:
len(states)

33

In [50]:
len(lang_states)

35

In [74]:
state_map = {}

count = 0
for state in states:
  state = state.split('_ln.csv')[0]
  actual = state
  if state == 'jk':
    state = 'jammu and kashmir'
  if '_' in state:
    state = state.replace('_', ' ')
  for st in lang_states:
    if state in st:
      state_map[actual] = st
      count = count + 1
print(count)

34


In [75]:
state_map

{'andhra': 'andhra pradesh',
 'arunachal': 'arunachal pradesh',
 'dadra': 'dadra and nagar haveli and daman and diu',
 'daman': 'dadra and nagar haveli and daman and diu',
 'delhi': 'delhi',
 'goa': 'goa',
 'jk': 'jammu and kashmir and ladakh',
 'manipur': 'manipur',
 'meghalaya': 'meghalaya',
 'mizoram': 'mizoram',
 'nagaland': 'nagaland',
 'puducherry': 'puducherry',
 'sikkim': 'sikkim',
 'kerala': 'kerala',
 'andaman': 'andaman and nicobar islands',
 'tripura': 'tripura',
 'himachal_pradesh': 'himachal pradesh',
 'uttarakhand': 'uttarakhand',
 'assam': 'assam',
 'karnataka': 'karnataka',
 'telangana': 'telangana',
 'jharkhand': 'jharkhand',
 'haryana': 'haryana',
 'punjab': 'punjab',
 'gujarat': 'gujarat',
 'chandigarh': 'chandigarh',
 'odisha': 'odisha',
 'madhya_pradesh': 'madhya pradesh',
 'rajasthan': 'rajasthan',
 'west_bengal': 'west bengal',
 'tamil_nadu': 'tamil nadu',
 'bihar': 'bihar',
 'uttar_pradesh': 'uttar pradesh'}

In [81]:
# create a new df
newdf = pd.DataFrame(columns=['state', 'first','' 'second', 'third', 'fourth'])

for state in state_map:
  print(state)
  temp = df[df['state'].str.lower() == state_map[state]]
  # add new row in newdf
  values = {'state': [state],
            'first': [temp['most_spoken_lang'].values[0].lower()],
            'second': [temp['second_most_spoken_lang'].values[0].lower()],
            'third': [temp['third_most_spoken_lang'].values[0].lower()],
            'fourth': [temp['fourth_most_spoken_lang'].values[0].lower()]
            }
  newdf = pd.concat([newdf, pd.DataFrame(values)], ignore_index=True) # Use pd.concat to add a row

andhra
arunachal
dadra
daman
delhi
goa
jk
manipur
meghalaya
mizoram
nagaland
puducherry
sikkim
kerala
andaman
tripura
himachal_pradesh
uttarakhand
assam
karnataka
telangana
jharkhand
haryana
punjab
gujarat
chandigarh
odisha
madhya_pradesh
rajasthan
west_bengal
tamil_nadu
bihar
uttar_pradesh


In [82]:
newdf

,state,first,second,third,fourth
0,andhra,telugu,urdu,tamil,chenchu
1,arunachal,nishi,adi,apatani,tagin
2,dadra,gujarati,hindi,marathi,konkani
3,daman,gujarati,hindi,marathi,konkani
4,delhi,hindi,punjabi,urdu,bengali
5,goa,konkani,marathi,hindi,kannada
6,jk,kashmiri,dogri,hindi,punjabi
7,manipur,meitei,nepali,hindi,bengali
8,meghalaya,khasi,garo,bengali,nepali
9,mizoram,mizo,english,hindi,meitei


In [89]:
# in the first column if regex "hindi (including.*) exists then replace it with hindi
newdf['first'] = newdf['first'].str.replace(r'hindi .including.*', 'hindi', regex=True)

In [92]:
newdf.head()

,state,first,second,third,fourth
0,andhra,telugu,urdu,tamil,chenchu
1,arunachal,nishi,adi,apatani,tagin
2,dadra,gujarati,hindi,marathi,konkani
3,daman,gujarati,hindi,marathi,konkani
4,delhi,hindi,punjabi,urdu,bengali


In [91]:
merged_df.head()

,last_name,state,count
0,shaik,andhra,423139.0
1,reddy,andhra,165388.0
2,rao,andhra,102433.0
3,areddy,andhra,49538.0
4,dasari,andhra,39064.0


In [93]:
# merge df and newdf based on state
merged_df = merged_df.merge(newdf, on='state', how='left')

In [94]:
merged_df.head()

,last_name,state,count,first,second,third,fourth
0,shaik,andhra,423139.0,telugu,urdu,tamil,chenchu
1,reddy,andhra,165388.0,telugu,urdu,tamil,chenchu
2,rao,andhra,102433.0,telugu,urdu,tamil,chenchu
3,areddy,andhra,49538.0,telugu,urdu,tamil,chenchu
4,dasari,andhra,39064.0,telugu,urdu,tamil,chenchu


In [95]:
merged_df[merged_df['last_name'] == 'shaik']

,last_name,state,count,first,second,third,fourth
0,shaik,andhra,423139.0,telugu,urdu,tamil,chenchu
362990,shaik,daman,1.0,gujarati,hindi,marathi,konkani
368951,shaik,delhi,83.0,hindi,punjabi,urdu,bengali
406546,shaik,goa,1245.0,konkani,marathi,hindi,kannada
452773,shaik,meghalaya,5.0,khasi,garo,bengali,nepali
463557,shaik,puducherry,98.0,tamil,telugu,malayalam,french
473532,shaik,kerala,170.0,malayalam,tamil,tulu,kannada
505115,shaik,andaman,5.0,bengali,hindi,tamil,telugu


In [96]:
all_langs = newdf['first'].tolist() + newdf['second'].tolist() + newdf['third'].tolist() + newdf['fourth'].tolist()
# remove duplicates in all_langs
all_langs = list(set(all_langs))

In [97]:
all_langs

['sindhi',
 'nepali',
 'kannada',
 'marathi',
 'mizo',
 'adi',
 'garo',
 'tagin',
 'assamese',
 'hindi',
 'odia',
 'french',
 'punjabi',
 'naga languages',
 'english',
 'chenchu',
 'urdu',
 'bengali',
 'maithili',
 'dogri',
 'kokborok',
 'santali',
 'kashmiri',
 'gujarati',
 'apatani',
 'tulu',
 'konkani',
 'telugu',
 'malayalam',
 'tamil',
 'meitei',
 'khasi',
 'gondi',
 'bodo',
 'nishi',
 'chakma',
 'pahari and kumauni']

In [98]:
# add all above columns to merged_df
for lang in all_langs:
  merged_df[lang] = 0

In [99]:
merged_df.head()

,last_name,state,count,first,second,third,fourth,sindhi,nepali,kannada,...,telugu,malayalam,tamil,meitei,khasi,gondi,bodo,nishi,chakma,pahari and kumauni
0,shaik,andhra,423139.0,telugu,urdu,tamil,chenchu,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,reddy,andhra,165388.0,telugu,urdu,tamil,chenchu,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,rao,andhra,102433.0,telugu,urdu,tamil,chenchu,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,areddy,andhra,49538.0,telugu,urdu,tamil,chenchu,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,dasari,andhra,39064.0,telugu,urdu,tamil,chenchu,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [100]:
merged_df.to_csv('all_states_with_languages.csv', index=False)

In [101]:
# based on first, second, third, fourth
# take count and assign 50% to first, 25% to second, 12.5 to third, 12.5 to fourth
def assign_weights(row_df):
  lang = row_df['first']
  row_df[lang] = 0.5 * row_df['count']
  lang = row_df['second']
  row_df[lang] = 0.25 * row_df['count']
  lang = row_df['third']
  row_df[lang] = 0.125 * row_df['count']
  lang = row_df['fourth']
  row_df[lang] = 0.125 * row_df['count']
  return row_df


merged_df = merged_df.apply(assign_weights, axis=1)

In [102]:
merged_df.head()

,last_name,state,count,first,second,third,fourth,sindhi,nepali,kannada,...,telugu,malayalam,tamil,meitei,khasi,gondi,bodo,nishi,chakma,pahari and kumauni
0,shaik,andhra,423139.0,telugu,urdu,tamil,chenchu,0.0,0.0,0.0,...,211569.5,0.0,52892.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,reddy,andhra,165388.0,telugu,urdu,tamil,chenchu,0.0,0.0,0.0,...,82694.0,0.0,20673.500,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,rao,andhra,102433.0,telugu,urdu,tamil,chenchu,0.0,0.0,0.0,...,51216.5,0.0,12804.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,areddy,andhra,49538.0,telugu,urdu,tamil,chenchu,0.0,0.0,0.0,...,24769.0,0.0,6192.250,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,dasari,andhra,39064.0,telugu,urdu,tamil,chenchu,0.0,0.0,0.0,...,19532.0,0.0,4883.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
merged_df.to_csv('all_states_with_languages.csv', index=False)

In [104]:
# drop state, count, first, second, third, fourth columns
final_df = merged_df.drop(['state', 'count', 'first', 'second', 'third', 'fourth'], axis=1)

In [106]:
final_df.head()

,last_name,sindhi,nepali,kannada,marathi,mizo,adi,garo,tagin,assamese,...,telugu,malayalam,tamil,meitei,khasi,gondi,bodo,nishi,chakma,pahari and kumauni
0,shaik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,211569.5,0.0,52892.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,reddy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,82694.0,0.0,20673.500,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,rao,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,51216.5,0.0,12804.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,areddy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,24769.0,0.0,6192.250,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,dasari,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,19532.0,0.0,4883.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [109]:
final_df[final_df['last_name'] == 'shaik']

,last_name,sindhi,nepali,kannada,marathi,mizo,adi,garo,tagin,assamese,...,telugu,malayalam,tamil,meitei,khasi,gondi,bodo,nishi,chakma,pahari and kumauni
0,shaik,0.0,0.000,0.000,0.000,0.0,0.0,0.00,0.0,0.0,...,211569.500,0.00,52892.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0
362990,shaik,0.0,0.000,0.000,0.125,0.0,0.0,0.00,0.0,0.0,...,0.000,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
368951,shaik,0.0,0.000,0.000,0.000,0.0,0.0,0.00,0.0,0.0,...,0.000,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
406546,shaik,0.0,0.000,155.625,311.250,0.0,0.0,0.00,0.0,0.0,...,0.000,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
452773,shaik,0.0,0.625,0.000,0.000,0.0,0.0,1.25,0.0,0.0,...,0.000,0.00,0.000,0.0,2.5,0.0,0.0,0.0,0.0,0.0
463557,shaik,0.0,0.000,0.000,0.000,0.0,0.0,0.00,0.0,0.0,...,24.500,12.25,49.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
473532,shaik,0.0,0.000,21.250,0.000,0.0,0.0,0.00,0.0,0.0,...,0.000,85.00,42.500,0.0,0.0,0.0,0.0,0.0,0.0,0.0
505115,shaik,0.0,0.000,0.000,0.000,0.0,0.0,0.00,0.0,0.0,...,0.625,0.00,0.625,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [112]:
# Automatically create the dictionary for aggregation
agg_dict = {col: 'sum' for col in final_df.columns if col != 'last_name'}

In [114]:
# Group by 'last_name' and aggregate
aggregated_df = final_df.groupby('last_name').agg(agg_dict).reset_index()

In [115]:
aggregated_df[aggregated_df['last_name'] == 'shaik']

,last_name,sindhi,nepali,kannada,marathi,mizo,adi,garo,tagin,assamese,...,telugu,malayalam,tamil,meitei,khasi,gondi,bodo,nishi,chakma,pahari and kumauni
768623,shaik,0.0,0.625,176.875,311.375,0.0,0.0,1.25,0.0,0.0,...,211594.625,97.25,52984.5,0.0,2.5,0.0,0.0,0.0,0.0,0.0


In [117]:
# make last_name all small case
aggregated_df['last_name'] = aggregated_df['last_name'].str.lower()

In [118]:
aggregated_df.head()

,last_name,sindhi,nepali,kannada,marathi,mizo,adi,garo,tagin,assamese,...,telugu,malayalam,tamil,meitei,khasi,gondi,bodo,nishi,chakma,pahari and kumauni
0,aadhumull,0.000,0.0,0.5,0.00,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0
1,bachhar,0.500,0.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,bachhodiya,0.125,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,bachhole,0.125,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,balait,0.125,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [119]:
aggregated_df.to_csv('all_states_with_languages_agg.csv', index=False)

In [120]:
aggregated_df.shape

(861132, 38)

In [121]:
aggregated_df.columns

Index(['last_name', 'sindhi', 'nepali', 'kannada', 'marathi', 'mizo', 'adi',
       'garo', 'tagin', 'assamese', 'hindi', 'odia', 'french', 'punjabi',
       'naga languages', 'english', 'chenchu', 'urdu', 'bengali', 'maithili',
       'dogri', 'kokborok', 'santali', 'kashmiri', 'gujarati', 'apatani',
       'tulu', 'konkani', 'telugu', 'malayalam', 'tamil', 'meitei', 'khasi',
       'gondi', 'bodo', 'nishi', 'chakma', 'pahari and kumauni'],
      dtype='object')